In [2]:
import os
import time
import numpy as np
import pickle
import matplotlib.pyplot as plt
import loader
import torch
from torch import optim
from torch import nn
from config import config_base
from config import config_r_net
from config import config_match_lstm
from config import config_match_lstm_plus
from config import config_qa_net
from config import config_bi_daf
from config import config_m_reader
from config import config_m_reader_plus
import preprocess_data
import utils
from modules.layers.loss import MyNLLLoss
from modules.layers.loss import RougeLoss
from modules import match_lstm
from modules import match_lstm_plus
from modules import r_net
from modules import bi_daf
from modules import qa_net
from preprocess_data import *
from modules import m_reader
from modules import m_reader_plus


# config = config_match_lstm.config
# config = config_match_lstm_plus.config
# config = config_r_net.config
# config = config_bi_daf.config
# config = config_qa_net.config
# config = config_m_reader.config
config = config_m_reader_plus.config

In [3]:
df = organize_data(config.train_data)
df = df.sample(300)

In [4]:
df = deal_data(df)
collect_data(df)
gen_vocab()
gen_tag_index(df)
gen_w2v()
build_embedding(config.vocab_path, config.my_embedding, config.embedding_path)

vacab length: 16907


2018-09-10 16:02:06,997 : INFO : collecting all words and their counts
2018-09-10 16:02:06,998 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-10 16:02:07,036 : INFO : collected 16904 word types from a corpus of 209932 raw words and 900 sentences
2018-09-10 16:02:07,036 : INFO : Loading a fresh vocabulary
2018-09-10 16:02:07,055 : INFO : min_count=1 retains 16904 unique words (100% of original 16904, drops 0)
2018-09-10 16:02:07,055 : INFO : min_count=1 leaves 209932 word corpus (100% of original 209932, drops 0)
2018-09-10 16:02:07,086 : INFO : deleting the raw counts dictionary of 16904 items
2018-09-10 16:02:07,087 : INFO : sample=0.001 downsamples 27 most-common words
2018-09-10 16:02:07,087 : INFO : downsampling leaves estimated 175735 word corpus (83.7% of prior 209932)
2018-09-10 16:02:07,116 : INFO : estimated required memory for 16904 words and 300 dimensions: 49021600 bytes
2018-09-10 16:02:07,117 : INFO : resetting layer weights


word flag num:64


2018-09-10 16:02:07,261 : INFO : training model with 3 workers on 16904 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-09-10 16:02:07,394 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:07,398 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:07,398 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:07,399 : INFO : EPOCH - 1 : training on 209932 raw words (175741 effective words) took 0.1s, 1282793 effective words/s
2018-09-10 16:02:07,547 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:07,554 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:07,560 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:07,560 : INFO : EPOCH - 2 : training on 209932 raw words (175829 effective words) took 0.2s, 1106204 effective words/s
2018-09-10 16:02:07,688 : INFO :

2018-09-10 16:02:10,083 : INFO : EPOCH - 20 : training on 209932 raw words (175831 effective words) took 0.1s, 1356388 effective words/s
2018-09-10 16:02:10,219 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:10,223 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:10,224 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:10,224 : INFO : EPOCH - 21 : training on 209932 raw words (175687 effective words) took 0.1s, 1278782 effective words/s
2018-09-10 16:02:10,348 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:10,353 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:10,354 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:10,354 : INFO : EPOCH - 22 : training on 209932 raw words (175638 effective words) took 0.1s, 1361158 effective words/s
2018-09-10 16:02:10,477 : INFO : worker thr

2018-09-10 16:02:12,751 : INFO : EPOCH - 40 : training on 209932 raw words (175755 effective words) took 0.1s, 1361244 effective words/s
2018-09-10 16:02:12,876 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:12,880 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:12,881 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:12,881 : INFO : EPOCH - 41 : training on 209932 raw words (175854 effective words) took 0.1s, 1365149 effective words/s
2018-09-10 16:02:13,020 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-10 16:02:13,027 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-10 16:02:13,031 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-10 16:02:13,032 : INFO : EPOCH - 42 : training on 209932 raw words (175548 effective words) took 0.1s, 1180341 effective words/s
2018-09-10 16:02:13,154 : INFO : worker thr

non-known word: <pad>
non-known word: <unk>
non-known word:  
embedding size is (16907, 300)


In [5]:
df = shorten_content_all(df, config.max_len)
df = build_answer_range(df)
df = select_data(df)
df_train, df_val, df_test = split_dataset(df)
df_train.to_csv(config.train_df, index=False)
df_val.to_csv(config.val_df, index=False)
df_test.to_csv(config.test_df, index=False)

答案存在比例：0.9833
截取比例：0.9833
截取准确率：1.0000
max length: 499
min length: 32
mean length:273
median length:265
answer generation accuracy(all): 0.9966
wrong split: 0.0034
标题和问题重复的数据：0.0000
问题和答案重复的数据:0.0000
当问题是标题类问题时，答案与标题不同的数据:0.0000
select data:1.0000
all data size:300
train size:270, shorten train size:264
val size:30, shorten val size:30


In [7]:
embedding_np = loader.load_w2v(config.embedding_path)

In [8]:
    if os.path.isfile(config.train_pkl):
        with open(config.train_pkl, 'rb') as file:
            train_data = pickle.load(file)
    else:
        train_data = loader.load_data(config.train_df, config.vocab_path, config.tag_path)
        with open(config.train_pkl, 'wb') as file:
            pickle.dump(train_data, file)

In [9]:
train_data[0]

[[271,
  2372,
  8770,
  7155,
  177,
  8771,
  1124,
  4147,
  8772,
  8773,
  8774,
  2419,
  2345,
  29,
  271,
  8775,
  57,
  877,
  128,
  2453,
  19,
  2256,
  56,
  1410,
  1767,
  8776,
  60,
  271,
  2372,
  8770,
  4840,
  61,
  1353,
  6893,
  177,
  8777,
  877,
  128,
  287,
  19,
  1124,
  4147,
  8772,
  8773,
  8774,
  2419,
  25,
  2345,
  34,
  678,
  109,
  4763,
  2835,
  7654,
  933,
  2419,
  372,
  2313,
  2835,
  8778,
  933,
  2419,
  34,
  8779,
  67,
  8780,
  1003,
  1006,
  29,
  108,
  1059,
  1136,
  94,
  271,
  8781,
  61,
  2553,
  291,
  84,
  25,
  5221,
  753,
  34,
  3306,
  3658,
  291,
  165,
  4414,
  291,
  165,
  867,
  372,
  1225,
  290,
  8041,
  291,
  29,
  271,
  2372,
  8770,
  4840,
  8782,
  6767,
  1560,
  89,
  8777,
  3163,
  3164,
  8783,
  2474,
  8784,
  63,
  61,
  2345,
  84,
  8785,
  29,
  2,
  8777,
  3163,
  3164,
  8783,
  2474,
  8784,
  63,
  1421,
  931,
  1318,
  8786,
  161,
  2130,
  271,
  8787,
  355,
  2819,
  2

In [6]:
300*0.0034

1.02